In [15]:
# import libaries

import csv
import json

In [ ]:
# Function: write_json
# Write a JSON file
# Purpose: Write a file from a dictonary in JSON format
# Input: Path to write out file, A list of dictonaries
# Output: A JSON formated file
# Restrictions: Writes the file with forced UTF-8 encoding

def write_json(file_path, list_dict):
    with open(file_path, 'w', encoding="utf8") as file_out:
        json.dump(list_dict, file_out)
    print(f'JSON file written to {file_out}')

In [24]:
# Function: read_csv
# Read a CSV file
# Purpose: Intake data that is in .csv format
# Input: Path to a singular file
# Output: Return a list of dictionaries of read CSV
# Restrictions: reads the file with forces UTF-8 encoding

def read_csv(file_path):
    with open(file_path, 'r', encoding="utf8") as file_in:
        file = csv.DictReader(file_in, delimiter=',')
        data_s0 = []
        for row in file:
            data_s0.append(row)
        data = data_s0
    return data

In [78]:
# Function: read_json
# Read a JSON file
# Purpose: Intake data that is in .json format
# Input: Path to a singular file
# Output: Return a list of dictionaries of read JSON
# Restrictions: reads the file with forces UTF-8 encoding

def read_json(file_path):
    with open(file_path, 'r', encoding="utf8") as file_in:
        data_s0 = json.load(file_in)
        data = data_s0
    return data

In [164]:
# Function: search_filter
# Filter a JSON file
# Purpose: Filter a JSON file based on a search, and return a filtered JSON
# Input: A search JSON
# Output: A JSON formatted results file and console display 
# Restrictions: (blank)

def search_fliter(search_in):
    choice_data = search_in['data']
    choice_field = search_in['field']
    choice_value = search_in['value']
    choice_criteria = search_in['criteria']
    file_path_in = f'json_files/data/{choice_data}.json'
    json_in = read_json(file_path_in)
    if choice_criteria == 
    filtered_s0 = filter(lambda f: f[choice_field] == choice_value, json_in)
    filtered = list(filtered_s0)
    file_path_out = f'json_files/output/{choice_data}_results.json'
    write_json(file_path_out, filtered)


In [165]:
search_fliter(read_json('json_files/search/search.json'))

JSON file written to <_io.TextIOWrapper name='json_files/output/sets_results.json' mode='w' encoding='utf8'>
sets
set_num
001-1
equals to
[]


In [82]:
# Function: search
# Search
# Purpose: filter JSON based on search.json
# Input: Expects criteria to be passed, if not looks for search.json in default location, 
#        if not found asks if search should be created 
# Output: A JSON formmated file that can be used by the search() funcation 
# Restrictions: (blank)

def search(choice_data = 'NE', choice_field = 'NE', choice_value= 'NE', choice_criteria = 'NE'):
    try:
        json_in = read_json('json_files/search/search.json')
        print(json_in)
    except:
        print("Cannot find search.json")
        choice_s = int(input("""
    Would you like to create a search?:
        1. Yes
        2. No
        
        """))
        if choice_s == 1:
            create_search()
            search()
        else:
            print("Goodbye")

In [132]:
create_search()

 
    Which data do you want to search (get records from): 
        1. sets
    
    1

    Do you want to filter "sets" 
        1. Yes
        2. No
    
    1
 
    Which field do you want to search in "sets" data: 
        1. set_num

        1
 
    What is the value you are searching for in field: "set_num" in "sets" data:

        001-1
 
    For value: "001-1", What is the criteria for searching for in field: "set_num" in "sets" data:
        1. equal to
        2. less than
        3. greater than
        4. all

        1
Searching for data that is equals to "001-1" in field: "set_num" from "sets" data
JSON file written to <_io.TextIOWrapper name='json_files/search/search.json' mode='w' encoding='utf8'>
{'data': 'sets', 'field': 'set_num', 'value': '001-1', 'criteria': 'equals to'}
<class 'dict'>
